In [26]:
import json 
import pandas as pd
import sqlalchemy as db

In [27]:
members = []
with open('senators.json') as f:
    for line in f:
        members.append(json.loads(line)) 

In [28]:
all_bills = []
with open('bills.json') as f:
    for line in f:
        all_bills.append(json.loads(line))  

In [29]:
votes = []
with open('votes.json') as f:
    for line in f:
        votes.append(json.loads(line))  

#### JSON wrangling for later sql insertion

In [5]:
keys = [(str(i.keys())) for i in votes]
key_list = [i[12:15] for i in keys]

In [6]:
vote_list = []
for i in range(0,len(key_list)):
    lst = votes[i][key_list[i]]
    for x in lst:
        d = {'roll_call': key_list[i], 'member_id' :x['member_id'], 'vote': x['vote']}
        vote_list.append(d)      

#### SQL Implementation

In [8]:
# Create Database 
engine = db.create_engine('sqlite:///congress_db.sqlite') 
connection = engine.connect()
metadata = db.MetaData()

In [9]:
#Create Member Table
senators = db.Table('senators', metadata,
                    db.Column('id',db.String(), nullable =False),
                    db.Column('first_name',db.String()),
                    db.Column('last_name',db.String()),
                    db.Column('gender',db.String()),
                    db.Column('party',db.String()),
                    db.Column('twitter_account',db.String()),
                    db.Column('state',db.String()))            
metadata.create_all(engine)

In [10]:
#Create Bill Table 
bills = db.Table ('bills', metadata, 
                db.Column('id',db.String()),
                db.Column('chamber',db.String()),
                db.Column('roll_call',db.Integer()),
                db.Column('title',db.String()),
                db.Column('date',db.String),
                db.Column('result',db.Integer()),
                db.Column('d_yes',db.Integer()),
                db.Column('d_no',db.Integer()),
                db.Column('r_yes',db.Integer()),
                db.Column('r_no',db.Integer()),
                db.Column('i_yes',db.Integer()),
                db.Column('i_no',db.Integer())
                )
metadata.create_all(engine)

In [11]:
votes = db.Table('votes', metadata,
                 db.Column('roll_call',db.Integer()),
                 db.Column('member_id',db.String()),
                 db.Column('vote',db.String()),
                )
metadata.create_all(engine)

In [12]:
metadata.tables.keys()

dict_keys(['senators', 'bills', 'votes'])

In [13]:
#insert data into senators table
query = db.insert(senators)
ResultProxy = connection.execute(query,members)

In [14]:
# check if data has been inserted into senator table
members_table = connection.execute(db.select([senators])).fetchall()

In [15]:
# insert data into bills table
query1 = db.insert(bills)
ResultProxy = connection.execute(query1,all_bills)


In [16]:
bills_table = connection.execute(db.select([bills])).fetchall()

In [17]:
# Insert data into vote table
votes1 = vote_list
query2 = db.insert(votes)
ResultProxy = connection.execute(query2,votes1)

In [18]:
votes_table = connection.execute(db.select([votes])).fetchall()

#### SQL Queries 

In [19]:
from sqlalchemy.sql import select
from sqlalchemy.sql import and_, or_, not_
conn = engine.connect()

In [20]:
#1) Twitter accounts of all senators
s = select([senators.c.first_name,senators.c.last_name, senators.c.twitter_account])
results = conn.execute(s)
df_twitter = pd.DataFrame(results)
df_twitter.columns = results.keys()
df_twitter.head()

,first_name,last_name,twitter_account
0,Lamar,Alexander,SenAlexander
1,Tammy,Baldwin,SenatorBaldwin
2,John,Barrasso,SenJohnBarrasso
3,Michael,Bennet,SenBennetCo
4,Richard,Blumenthal,SenBlumenthal


In [21]:
#2) Return the voting results of all Senators for all bills
query = db.select([senators.c.first_name, senators.c.last_name,votes.c.roll_call, votes.c.vote, 
                   bills.c.id, bills.c.result])
query = query.where(and_(bills.c.roll_call == votes.c.roll_call, votes.c.member_id == senators.c.id ))
results = connection.execute(query)
df_all_bills = pd.DataFrame(results)
df_all_bills.columns = results.keys()
df_all_bills.head(5)   

,first_name,last_name,roll_call,vote,id,result
0,Lamar,Alexander,160,Yes,hres294-116,Passed
1,Lamar,Alexander,161,Yes,hres294-116,Passed
2,Lamar,Alexander,162,Yes,hr1759-116,Passed
3,Lamar,Alexander,163,Yes,hr1644-116,Agreed to
4,Lamar,Alexander,164,Yes,hr1644-116,Agreed to


In [22]:
# 3) All the senators that have voted 'Yes' for roll_call 167
query = db.select([senators.c.first_name, senators.c.last_name,votes.c.roll_call, votes.c.vote, bills.c.id, bills.c.result])
query = query.where(and_(bills.c.roll_call == votes.c.roll_call, votes.c.member_id == senators.c.id, votes.c.roll_call ==167, votes.c.vote == 'Yes' ))
results = connection.execute(query)
df_bill160_yes = pd.DataFrame(results)
df_bill160_yes.columns = results.keys()
df_bill160_yes.head()

,first_name,last_name,roll_call,vote,id,result
0,Lamar,Alexander,167,Yes,hr1644-116,Passed
1,John,Barrasso,167,Yes,hr1644-116,Passed
2,Roy,Blunt,167,Yes,hr1644-116,Passed
3,John,Boozman,167,Yes,hr1644-116,Passed
4,Richard,Burr,167,Yes,hr1644-116,Passed


In [23]:
# 4) All the senators that have voted 'No' for roll_call 167
query = db.select([senators.c.first_name, senators.c.last_name,votes.c.roll_call, votes.c.vote, bills.c.id, bills.c.result])
query = query.where(and_(bills.c.roll_call == votes.c.roll_call, votes.c.member_id == senators.c.id, votes.c.roll_call ==167, votes.c.vote == 'No' ))
results = connection.execute(query)
df_bill160_no = pd.DataFrame(results)
df_bill160_no.columns = results.keys()
df_bill160_no.head()

,first_name,last_name,roll_call,vote,id,result
0,Tammy,Baldwin,167,No,hr1644-116,Passed
1,Michael,Bennet,167,No,hr1644-116,Passed
2,Richard,Blumenthal,167,No,hr1644-116,Passed
3,Cory,Booker,167,No,hr1644-116,Passed
4,Sherrod,Brown,167,No,hr1644-116,Passed


In [24]:
#5) All the senators that have voted 'No' for roll_call 160
query = db.select([senators.c.first_name, senators.c.last_name,votes.c.roll_call, 
                   votes.c.vote, bills.c.id, bills.c.result])
query = query.where(and_(bills.c.roll_call == votes.c.roll_call, 
                         votes.c.member_id == senators.c.id, votes.c.roll_call ==160, votes.c.vote == 'No'))
results = connection.execute(query)
df_bill160_no = pd.DataFrame(results)
df_bill160_no.columns = results.keys()
df_bill160_no

,first_name,last_name,roll_call,vote,id,result
0,Cory,Booker,160,No,hres294-116,Passed
1,Sherrod,Brown,160,No,hres294-116,Passed
2,Kirsten,Gillibrand,160,No,hres294-116,Passed
3,Kamala,Harris,160,No,hres294-116,Passed
4,Martin,Heinrich,160,No,hres294-116,Passed
5,Mazie,Hirono,160,No,hres294-116,Passed
6,Jeff,Merkley,160,No,hres294-116,Passed
7,Gary,Peters,160,No,hres294-116,Passed
8,Bernard,Sanders,160,No,hres294-116,Passed
9,Debbie,Stabenow,160,No,hres294-116,Passed


In [25]:
#6) Return all the bills Luther Strange voted "Yes" for
query = db.select([senators.c.first_name, senators.c.last_name, bills.c.id])
query = query.where(and_(bills.c.roll_call == votes.c.roll_call, 
        votes.c.member_id == senators.c.id, senators.c.last_name == 'Strange', votes.c.vote == 'Yes'))
results = connection.execute(query)
df_Strange = pd.DataFrame(results)
df_Strange.columns = results.keys()
df_Strange


,first_name,last_name,id
0,Luther,Strange,hres294-116
1,Luther,Strange,hres294-116
2,Luther,Strange,hr1759-116
3,Luther,Strange,hr1644-116
4,Luther,Strange,hr1644-116
5,Luther,Strange,hr1644-116
6,Luther,Strange,hr1644-116
7,Luther,Strange,hr1644-116
